### The code snippets below will migrate datasources followed by workbooks from source to target Tableau Server.

In [ ]:
import tableauserverclient as TSC
import getpass
import pandas as pd
import json

In [ ]:
# Connect to Source Server

source_ts = '<<Source Server>>'
source_user = '<<Tableau Admin User>>'
source_site = '<<Tableau Online Site>>' # For Tablesu Online Only

source_pwd = getpass.getpass("Enter password of source Tableau Server: ")
source_tableau_auth = TSC.TableauAuth(source_user, source_pwd, source_site)
source_server = TSC.Server(source_ts,use_server_version=True)
with source_server.auth.sign_in(source_tableau_auth): 
    print("Connected to Source Server")

In [ ]:
# Connect to Target Server

target_ts = '<<Target Server>>'
target_user = '<<Tableau Admin User>>'
target_site = '<<Tableau Online Site>>' # For Tablesu Online Only

target_pwd = getpass.getpass("Enter password of target Tableau Server: ")
target_tableau_auth = TSC.TableauAuth(target_user, target_pwd)
target_server = TSC.Server(target_ts,use_server_version=True)
with target_server.auth.sign_in(target_tableau_auth): 
    print("Connected to Target Server") 

In [ ]:
#Load Mapping Data 

datasource_project_mapping_df = pd.read_csv("datasource_target_project_mapping.csv",sep='|')
datasource_project_mapping = dict(zip(datasource_project_mapping_df.SourceDataSourceName,
                                      datasource_project_mapping_df.TargetProjectID))

workbook_project_mapping_df = pd.read_csv("workbook_target_project_mapping.csv", sep='|')
workbook_project_mapping = dict(zip(workbook_project_mapping_df.SourceWorkbookName, workbook_project_mapping_df.TargetProjectID))

user_groups_mapping_df = pd.read_csv("user_groups_mapping.csv", sep='|')

datasource_credentials_df = pd.read_csv("datasource_credentials.csv",sep='|')
datasource_credentials =  datasource_credentials_df.set_index('LiveDataSourceName').T.to_dict('list')

workbook_credentials_df = pd.read_csv("workbook_credentials.csv",sep='|')
workbook_credentials =  workbook_credentials_df.set_index('LiveWorkbookName').T.to_dict('list')


##### Migrating data sources - Below code migrate datasource published separately on the source server. 

In [ ]:
with source_server.auth.sign_in(source_tableau_auth):
    all_datasources, pagination_item = source_server.datasources.get()
   
    for datasource in all_datasources:
        #if datasource.project_name == 'TSMigrationSource':
        if datasource.name in datasource_project_mapping:
            ds_file_path = source_server.datasources.download(datasource.id)
            with target_server.auth.sign_in(target_tableau_auth):
                new_ds = TSC.DatasourceItem(datasource_project_mapping[datasource.name])
                try:
                    if datasource.name in datasource_credentials:
                        conn_creds = TSC.ConnectionCredentials(name=datasource_credentials['may20testdata (new_test_db)'][0],
                                                               password=datasource_credentials['may20testdata (new_test_db)'][1])
                        new_ds = target_server.datasources.publish(new_ds, ds_file_path, overwrite_true,
                                                                  connection_credentials=conn_creds)
                    else:
                        new_ds = target_server.datasources.publish(new_ds, ds_file_path, overwrite_true)
                    print(ds_file_path + ' Uploaded Successfully.')
                except:
                    print(ds_file_path + ' Not Uploaded\n')

##### Migrating workbooks - Below code migrate workbooks on the source server. 

In [ ]:
# Uploading workbooks 

successfully_uploaded = {} #Keep a list of successfully uploaded workbooks.

with source_server.auth.sign_in(source_tableau_auth): 
    all_workbooks, pagination_item = source_server.workbooks.get()
              
    for workbook in all_workbooks:
         if workbook.name in workbook_project_mapping:
            print("Workbok ==> " + workbook.name)
            wb_file_path = source_server.workbooks.download(workbook.id)
            with target_server.auth.sign_in(target_tableau_auth):
                new_wb = TSC.WorkbookItem(workbook_project_mapping[workbook.name])
                try:
                    if workbook.name in workbook_credentials:
                        conn_creds = TSC.ConnectionCredentials(name=workbook_credentials[workbook.name][0],
                                                               password=workbook_credentials[workbook.name][1])
                        new_wb = target_server.workbooks.publish(new_wb, wb_file_path, overwrite_true,
                                                                 connection_credentials=conn_creds)
                        print(wb_file_path + ' Uploaded Successfully.')
                    else:
                        new_wb = target_server.workbooks.publish(new_wb, wb_file_path, overwrite_true)
                        print(wb_file_path + ' Uploaded Successfully.')
                    successfully_uploaded[wb_file_path] = True
                except:
                    print(wb_file_path + ' Not Uploaded')
                    successfully_uploaded[wb_file_path] = False
                    
#print(successfully_uploaded)